In [280]:
from os import chdir
from google.colab import drive

drive.mount("/content/drive", force_remount=True)
chdir("/content/drive/MyDrive/Eliott/files/")

!pip install tweet-preprocessor

Mounted at /content/drive


In [328]:
# IMPORTS

import numpy as np
import pandas as pd
import preprocessing as pp
import json


from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline


from sklearn import preprocessing



#keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.layers as L
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam


import plotly.express as px



%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [329]:
file_name_train = 'Corona_NLP_train.csv'
file_name_test = 'Corona_NLP_test.csv'
X_train, y_train = pp.prepare_dataframe(file_name_train,lemmatising=False)
X_test, y_test = pp.prepare_dataframe(file_name_test,lemmatising=False)

On enlève : 
- Les URLS
- Hashtags
- Mentions
- Mots réservés
- Emojis et smileys

- 35525 mots sans lemmatisation
- 30794 avec lemmatisation

In [330]:
y_train

0        1
1        2
2        2
3        2
4        0
        ..
41152    1
41153    0
41154    2
41155    1
41156    0
Name: Sentiment_Number, Length: 41157, dtype: int64

In [331]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X = tokenizer.texts_to_sequences(X_train)

vocab_size = len(tokenizer.word_index)+1

print("Vocabulary size: {}".format(vocab_size))
print("\nExample:\n")
print("Sentence:\n{}".format(X_train[6]))
print("\nAfter tokenizing :\n{}".format(X[6]))

X = pad_sequences(X, padding='post')
print("\nAfter padding :\n{}".format(X[6]))
pad_len = X[0].shape[0]

Vocabulary size: 36701

Example:

Sentence:
Cashier at grocery store was sharing his insights on To prove his credibility he commented "I'm in Civics class so I know what I'm talking about".

After tokenizing :
[1208, 16, 26, 18, 70, 1304, 244, 816, 11, 2, 2927, 244, 9728, 183, 9729, 221, 6, 18100, 1509, 54, 13, 124, 61, 221, 946, 58]

After padding :
[ 1208    16    26    18    70  1304   244   816    11     2  2927   244
  9728   183  9729   221     6 18100  1509    54    13   124    61   221
   946    58     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]


In [332]:
pad_len

64

*Fancy* 

In [333]:
tf.keras.backend.clear_session()

# hyper parameters
EPOCHS = 5
BATCH_SIZE =  32 # 32
embedding_dim =  16
units = 256

model = tf.keras.Sequential([
    L.Embedding(vocab_size, embedding_dim, input_length=X.shape[1]),
    L.Bidirectional(L.LSTM(units,return_sequences=True)),
    L.GlobalMaxPool1D(),
    L.Dropout(0.4), # 0.4
    L.Dense(64, activation="relu"), #64
    L.Dropout(0.4), # 0.4 
    L.Dense(3)
])


# model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
#               optimizer='adam',metrics=['accuracy']
#              )

model.compile(loss="mean_squared_error",
              optimizer='adam',metrics=['accuracy']
             )

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 64, 16)            587216    
                                                                 
 bidirectional (Bidirectiona  (None, 64, 512)          559104    
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 512)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0

In [334]:
ohe = preprocessing.OneHotEncoder()
y_train2 = ohe.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_test2 = ohe.fit_transform(np.array(y_test).reshape(-1, 1)).toarray()


In [335]:
history = model.fit(X, y_train2, epochs=EPOCHS, validation_split=0.12, batch_size=BATCH_SIZE)


Epoch 1/5
1132/1132 [==============================] - 19s 14ms/step - loss: 0.1331 - accuracy: 0.7264 - val_loss: 0.0702 - val_accuracy: 0.8686
Epoch 2/5
1132/1132 [==============================] - 16s 14ms/step - loss: 0.0673 - accuracy: 0.8973 - val_loss: 0.0592 - val_accuracy: 0.8880
Epoch 3/5
1132/1132 [==============================] - 16s 14ms/step - loss: 0.0476 - accuracy: 0.9358 - val_loss: 0.0625 - val_accuracy: 0.8834
Epoch 4/5
1132/1132 [==============================] - 16s 14ms/step - loss: 0.0385 - accuracy: 0.9538 - val_loss: 0.0571 - val_accuracy: 0.8880
Epoch 5/5
1132/1132 [==============================] - 16s 14ms/step - loss: 0.0337 - accuracy: 0.9625 - val_loss: 0.0586 - val_accuracy: 0.8872


In [336]:
fig = px.line(
    history.history, y=['accuracy', 'val_accuracy'],
    labels={'index': 'epoch', 'value': 'accuracy'}
)

fig.show()

In [337]:
fig = px.line(
    history.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'loss'}
)

fig.show()

In [338]:
X_test[0]

'TRENDING: New Yorkers encounter empty supermarket shelves (pictured, Wegmans in Brooklyn), sold-out online grocers (FoodKick, MaxDelivery) as -fearing shoppers stock up'

In [339]:
vocab_size

36701

In [340]:
X_test = tokenizer.texts_to_sequences(X_test)


In [341]:

X_test = pad_sequences(X_test, padding='post',maxlen=pad_len) # on doit préciser la taille car ce n'est pas la même pour le train et le test

In [342]:
predict_x=model.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

In [344]:
loss, acc = model.evaluate(X_test,y_test2,verbose=1)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))

119/119 [==============================] - 1s 6ms/step - loss: 0.0602 - accuracy: 0.8797
Test loss: 0.060200005769729614
Test Accuracy: 0.8796735405921936


# Recherche d'Hyperparamètres

In [ ]:
pipeline_sgd = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', SGDClassifier()),
])

pipeline_gb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', GradientBoostingClassifier()),
])

pipeline_rf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', RandomForestClassifier()),
])

pipeline_lr = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LogisticRegression(max_iter=1000)),
])

pipeline_per = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', Perceptron()),
])

pipeline_svc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC(max_iter=10000)),
])


In [ ]:
models_and_params = {
    "SGD" : { "model" : pipeline_sgd,
              "params" : {
                    'clf__penalty': ['l2', 'l1', 'elasticnet'],
                    'clf__alpha': np.linspace(1e-10, 1e-4, 40),  
                        }
    },
    "GB" : { "model" : pipeline_gb,
              "params" : {
                    "clf__learning_rate": [0.1, 0.2,0.4,0.8],
                    "clf__n_estimators":[1600]#,3200]
                        }
    },
    "RF" : { "model" : pipeline_rf,
              "params" : {
                    'clf__n_estimators': [100, 200],   
                    'clf__max_depth': [300,600,None],
                    'clf__min_samples_leaf': [1, 2, 3],   
                    'clf__min_samples_split': [16, 32, 64],
                    'clf__max_features': ['log2', 'sqrt'],
                    'clf__criterion': ['gini', 'entropy']
                        }
    },
    "LR" : { "model" : pipeline_lr,
              "params" : {
                    'clf__C': [20,10, 1],
                    'clf__tol': np.linspace(1e-12,1e-6,20)
                        }
    },
    "PER" : { "model" : pipeline_per,
              "params" : {
                    'clf__penalty': ['l2', 'l1', 'elasticnet'],
                    'clf__alpha': np.linspace(1e-8, 1e-4, 100),
                        }
    },
    "SVC" : { "model" : pipeline_svc,
              "params" : {
                    'clf__penalty': ['l2', 'l1', 'elasticnet'],
                    'clf__loss': ['hinge', 'squared_hinge'],
                    'clf__dual' : [False,True]
                        }
    }
    

}

In [ ]:
def grid_Search(model_name,subset=-1):

    model = models_and_params[model_name]["model"]
    parameters = models_and_params[model_name]["params"]

    grid_clf = GridSearchCV(model, parameters,  scoring='accuracy', verbose=1 ,n_jobs=-1)
    
    if subset==-1:
        grid_clf.fit(X_train, y_train)
    else:
        grid_clf.fit(X_train[:subset], y_train[:subset])


    print("Best Score: ", grid_clf.best_score_)
    print("Best Params: ", grid_clf.best_params_)

    return grid_clf


### Saving Hyperparameters in JSON

We will first load old weights and updates only if needed. Then we can use the model with best params in the end.

In [ ]:
with open('data.json') as json_file:
    dico = json.load(json_file)

### SGD

In [ ]:
%%time
grid_sgd = grid_Search("SGD") 

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Best Score:  0.8807008312810172
Best Params:  {'clf__alpha': 1.28206e-05, 'clf__penalty': 'l1'}
CPU times: user 29 s, sys: 4.25 s, total: 33.2 s
Wall time: 7min 37s


In [ ]:
dico['SGD'] = grid_sgd.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

### Gradient boosting

In [ ]:
%%time
grid_gb = grid_Search("GB") 

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Score:  0.8391769441865644
Best Params:  {'clf__learning_rate': 0.4, 'clf__n_estimators': 1600}
CPU times: user 33min 38s, sys: 7.7 s, total: 33min 46s
Wall time: 3h 40min 28s


In [ ]:
dico['GB'] = grid_gb.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

### Random Forest

In [ ]:
%%time
grid_rf = grid_Search("RF") 

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best Score:  0.7047644196092446
Best Params:  {'clf__criterion': 'entropy', 'clf__max_depth': None, 'clf__max_features': 'log2', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 32, 'clf__n_estimators': 200}
CPU times: user 2min 29s, sys: 14 s, total: 2min 43s
Wall time: 2h 30min 23s


In [ ]:
dico['RF'] = grid_rf.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)


### Logistic Regression

In [ ]:
%%time
grid_lr = grid_Search("LR") 

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Score:  0.8230677619122002
Best Params:  {'clf__C': 10, 'clf__tol': 1e-12}
CPU times: user 1min 20s, sys: 2min 15s, total: 3min 35s
Wall time: 36min 42s


In [ ]:
dico['LR'] = grid_lr.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

### Perceptron

In [ ]:
%%time
grid_per = grid_Search("PER") 

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Best Score:  0.818353925564533
Best Params:  {'clf__alpha': 3.04e-06, 'clf__penalty': 'l1'}
CPU times: user 1min 10s, sys: 7.51 s, total: 1min 17s
Wall time: 16min 16s


In [ ]:
dico['Perceptron'] = grid_per.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

#### Observation 

La regression logistique n'est qu'un perceptron avec une sigmoid en fonction d'activation.
On voit que la Regression Logistique a de meilleures performances à l'issue de la recherche d'hyperparamètres mais pas de loin. Par ailleurs le temps d'entrainement est considérablement plus élevé pour la regression logistique (du au calcul de l'exponentiel). Nous verrons par la suite quel modèle il est préférable de conserver. 

### LinearSVC

In [ ]:
%%time
grid_svc = grid_Search("SVC") 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_classes.py", line 272, in fit
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sk

Best Score:  0.868916566574562
Best Params:  {'clf__dual': False, 'clf__loss': 'squared_hinge', 'clf__penalty': 'l1'}
CPU times: user 11.5 s, sys: 524 ms, total: 12.1 s
Wall time: 1min 7s


In [ ]:
dico['SVC'] = grid_svc.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

### Charging the best parameters for training

In [ ]:
# Opening JSON file
with open('data.json') as json_file:
    data = json.load(json_file)


data_sgd = data['SGD']
data_rf = data['RF']
data_gb = data['GB']
data_lr = data['LR']
data_per = data['Perceptron']
data_svc = data['SVC']



### Training on all training data and testing on test data

In [ ]:
def testing(model):
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)
    accuracy_train = accuracy_score(y_train,predictions_train )
    accuracy_test = accuracy_score(y_test,predictions_test )
    
    print(f"train_accuracy : {accuracy_train} \ntest_accuracy : {accuracy_test}  ")




### SGD

In [ ]:
model_sgd_best = pipeline_sgd
model_sgd_best.set_params(**data_sgd,clf__n_jobs=-1) 

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 SGDClassifier(alpha=1.28206e-05, n_jobs=-1, penalty='l1'))])

In [ ]:
%%time

"""
Si on a fait la recherche avant, on prend le best modèle,

Sinon, on reconstruit le modèle avec les best paramètres et on le fit à toutes les données d'entrainement. 

"""

model_sgd_best.fit(X_train,y_train)


CPU times: user 2.06 s, sys: 166 ms, total: 2.22 s
Wall time: 1.51 s


In [ ]:
testing(model_sgd_best)

train_accuracy : 0.9316762640620064 
test_accuracy : 0.8788836229594523  


### Random Forest

In [ ]:
model_rf_best = pipeline_rf
model_rf_best.set_params(**data_rf,clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(criterion='entropy',
                                        max_features='log2',
                                        min_samples_split=32, n_estimators=200,
                                        n_jobs=-1))])

In [ ]:
%%time

model_rf_best.fit(X_train,y_train)

CPU times: user 55.5 s, sys: 382 ms, total: 55.9 s
Wall time: 15.1 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(criterion='entropy',
                                        max_features='log2',
                                        min_samples_split=32, n_estimators=200,
                                        n_jobs=-1))])

In [ ]:
testing(model_rf_best)

train_accuracy : 0.9957965838132031 
test_accuracy : 0.7037914691943128  


### Gradient Boosting 

In [ ]:
model_gb_best = pipeline_gb
model_gb_best.set_params(**data_gb)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.4,
                                            n_estimators=1600))])

In [ ]:
%%time
model_gb_best.fit(X_train,y_train)

CPU times: user 29min 46s, sys: 4.29 s, total: 29min 51s
Wall time: 29min 43s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.4,
                                            n_estimators=1600))])

In [ ]:
testing(model_gb_best)

train_accuracy : 0.9979104405082975 
test_accuracy : 0.8457082675092154  


### Logistic Regression

In [ ]:
model_lr_best = pipeline_lr
model_lr_best.set_params(**data_lr,clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-12))])

In [ ]:
%%time

model_lr_best.fit(X_train,y_train)

CPU times: user 1.15 s, sys: 85 ms, total: 1.24 s
Wall time: 17.4 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-12))])

In [ ]:
testing(model_lr_best)

train_accuracy : 0.9755813105911509 
test_accuracy : 0.833070036861506  


### Perceptron

In [ ]:
model_per_best = pipeline_per
model_per_best.set_params(**data_per,clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', Perceptron(alpha=3.04e-06, n_jobs=-1, penalty='l1'))])

In [ ]:
%%time
model_per_best.fit(X_train,y_train)

CPU times: user 1.79 s, sys: 248 ms, total: 2.04 s
Wall time: 1.38 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', Perceptron(alpha=3.04e-06, n_jobs=-1, penalty='l1'))])

In [ ]:
testing(model_per_best)

train_accuracy : 0.941662414656073 
test_accuracy : 0.8217482885729331  


### SVC

In [ ]:
model_svc_best = pipeline_svc
model_svc_best.set_params(**data_svc)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

In [ ]:
%%time

model_svc_best.fit(X_train,y_train)

CPU times: user 7.69 s, sys: 31 ms, total: 7.72 s
Wall time: 7.69 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

In [ ]:
testing(model_svc_best)

train_accuracy : 0.9481983623684914 
test_accuracy : 0.8730911005792522  


### Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier


In [ ]:
clf1 = model_sgd_best

clf2 = model_rf_best

clf3 = model_gb_best

clf4 = model_lr_best

clf5 = model_per_best

clf6 = model_svc_best


eclf1 = VotingClassifier(
     estimators=[('sgd', clf1), ('rf', clf2), ('gb', clf3), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')

eclf2 = VotingClassifier(
     estimators=[('sgd', clf1), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')


In [ ]:
%%time

eclf1.fit(X_train,y_train)

CPU times: user 30min 13s, sys: 3.55 s, total: 30min 17s
Wall time: 30min 37s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SGDClassifier(alpha=1.28206e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('rf',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               RandomForestClassifier(criterion='entropy',
                                                                      max_features='log2',
                                                                      min_samples_split=32,
                                                                      n_estimators=200,
                                                                      n_j

In [ ]:
testing(eclf1)

train_accuracy : 0.9785698666083534 
test_accuracy : 0.8701948393891522  


In [ ]:
%%time

eclf2.fit(X_train,y_train)

CPU times: user 12.4 s, sys: 533 ms, total: 13 s
Wall time: 27.3 s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SGDClassifier(alpha=1.28206e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('lr',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               LogisticRegression(C=10,
                                                                  max_iter=1000,
                                                                  n_jobs=-1,
                                                                  tol=1e-12))])),
                             ('per',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
          

In [ ]:
testing(eclf2)

train_accuracy : 0.9609057997424496 
test_accuracy : 0.8654555028962612  


### Tests perso

In [ ]:
tests_perso = ["I think covid is the most horrible threat we ever faced",
                "I love covid, thanks to it I can see my family much more often and I don't have to comute as much",
                "I would love to come to your birthday party, but I got covid, I have to stay confined"
]

In [ ]:
predictions_out = eclf2.predict(tests_perso)


In [ ]:
predictions_out

array([-1,  1,  1])